In [2]:
# from pathlib import Path
# from time import time
# import math
# import json
# import csv
# import re
# import gc

# from scipy import signal
# import matplotlib.pyplot as plt
# import numpy as np
# import pandas as pd

# import mne
# import direct_neural_biasing as dnb

from brpylib import NsxFile
import os
import numpy as np
import matplotlib.pyplot as plt

## continuous data

In [3]:
data_file = 'data/Patient02.ns5'
mrk_file = 'data/Patient02_OfflineMrk.mrk'
nsx_file = NsxFile(data_file)

data = nsx_file.getdata()
nsx_file.close()


Patient02.ns5 opened

Patient02.ns5 closed


In [4]:
data

{'elec_ids': [1],
 'start_time_s': 0.0,
 'data_time_s': 2774.0,
 'downsample': 1,
 'data': array([[  0.  ,   0.  ,   0.  , ..., -69.  , -69.25, -69.5 ]],
       dtype=float32),
 'data_headers': [{'Header': 1, 'Timestamp': 0, 'NumDataPoints': 83220000}],
 'ExtendedHeaderIndices': [0],
 'samp_per_s': 30000.0}

In [17]:
np.shape(data['data'])

(1, 1, 83220000)

In [11]:
data_cont = data['data'][0][0]  # the [0][0] may be because we're sampling just channel 1 ?
t_offset = int(data['data_time_s'])

In [17]:
def plot_data_stream_section(data_stream, n, m):
    """
    Plots an n-m section of a 1D integer data stream as a line graph.

    Args:
        data_stream (list or numpy.ndarray): The 1D integer data stream.
        n (int): The starting index (inclusive).
        m (int): The ending index (inclusive).
    """
    section = data_stream[n : m + 1]
    indices = range(n, m + 1)

    plt.figure(figsize=(14, 6))
    plt.plot(indices, section)
    plt.title(f'Data Section {n} to {m}')
    plt.xlabel('Index')
    plt.ylabel('Value')
    plt.grid(True)
    plt.show()

start_t = 2515 + t_offset
end_t = 4515 + t_offset

plot_data_stream_section(data_cont, start_t, end_t)

IndexError: invalid index to scalar variable.

## .mrk file - offlineSW

In [20]:
# --- Concise .mrk file parser ---
def parse_mrk_file_concise(filepath):
    """
    Parses a .mrk file into a dictionary (signal_type: [indices]).
    Assumes first line is header, subsequent lines are 'index index signal_type'.
    """
    mrk_data = {}
    with open(filepath, 'r') as f:
        next(f) # Skip header line
        for line in f:
            parts = line.strip().split()
            if len(parts) == 3:
                index = int(parts[0])
                signal_type = parts[2]
                mrk_data.setdefault(signal_type, []).append(index)
    return mrk_data

def plot_marker_with_context(data_stream, marker_index, signal_type, context_window=500, output_dir="marker_plots"):
    """
    Plots a single marker with surrounding data context.
    """
    data_length = len(data_stream)
    plot_start = max(0, marker_index - context_window)
    plot_end = min(data_length - 1, marker_index + context_window)

    section_data = data_stream[plot_start : plot_end + 1]
    section_indices = range(plot_start, plot_end + 1)

    # CORRECTED LINE: Check if the section_data is empty using its length/size
    if len(section_data) == 0: # or if section_data.size == 0: if you're sure it's a numpy array
        print(f"Skipping plot for marker {marker_index} ({signal_type}) due to empty data section.")
        return

    plt.figure(figsize=(14, 6))
    plt.plot(section_indices, section_data, label='Continuous Data', color='blue', linewidth=1.5)

    # Highlight the marker point
    plt.axvline(x=marker_index, color='red', linestyle='--', label=f'Marker: {signal_type}')
    # Ensure the marker_index is within the bounds of data_stream before trying to access it
    if 0 <= marker_index < data_length:
        plt.plot(marker_index, data_stream[marker_index], 'ro', markersize=8, label='Marker Location')
    else:
        print(f"Warning: Marker index {marker_index} is out of bounds for data_stream. Cannot plot marker point.")


    plt.title(f'Signal Type: {signal_type} at Index: {marker_index} (Context: $\pm${context_window})')
    plt.xlabel('Data Index')
    plt.ylabel('Value')
    plt.grid(True, linestyle=':', alpha=0.7)
    plt.legend()
    plt.tight_layout()

    # Save the plot
    os.makedirs(output_dir, exist_ok=True) # Ensure output directory exists
    filename = os.path.join(output_dir, f"{signal_type}_marker_{marker_index}.png")
    plt.savefig(filename)
    plt.close()
    print(f"  Saved plot for marker {marker_index} as {filename}")

In [88]:
# --- Main Execution ---

# 1. Parse the .mrk file
parsed_markers = parse_mrk_file_concise(mrk_file)
print("\nParsed Markers:")
for signal_type, indices in parsed_markers.items():
    print(f"  {signal_type}: {indices[:5]}... ({len(indices)} total)")

# 2. Plot each marker with context
context_window_size = 500 # Adjust this to change how much data is shown around each marker
print(f"\nGenerating plots with context window of $\pm${context_window_size}...")
for signal_type, indices in parsed_markers.items():
    for marker_index in indices:
        plot_marker_with_context(data_cont, marker_index, signal_type, context_window=context_window_size)

print("\nAll marker plots generated in 'marker_plots' directory.")



Parsed Markers:
  offlineSW: [3515, 7210, 7897, 14431, 16326]... (208 total)

Generating plots with context window of $\pm$500...
  Saved plot for marker 3515 as marker_plots\offlineSW_marker_3515.png
  Saved plot for marker 7210 as marker_plots\offlineSW_marker_7210.png
  Saved plot for marker 7897 as marker_plots\offlineSW_marker_7897.png
  Saved plot for marker 14431 as marker_plots\offlineSW_marker_14431.png
  Saved plot for marker 16326 as marker_plots\offlineSW_marker_16326.png
  Saved plot for marker 22898 as marker_plots\offlineSW_marker_22898.png
  Saved plot for marker 34742 as marker_plots\offlineSW_marker_34742.png
  Saved plot for marker 41220 as marker_plots\offlineSW_marker_41220.png
  Saved plot for marker 67976 as marker_plots\offlineSW_marker_67976.png
  Saved plot for marker 104391 as marker_plots\offlineSW_marker_104391.png
  Saved plot for marker 110861 as marker_plots\offlineSW_marker_110861.png
  Saved plot for marker 120764 as marker_plots\offlineSW_marker_1207